## AI Agent Core

**Goal:** Build intelligent agents that generate, plan, and optimize content using foundation models (like GPT, open-source LLMs, etc.) through the **LangChain framework.**

Build **modular, agentic components** that:
| Step | Agent               | Output                                             |
| ---- | ------------------- | -------------------------------------------------- |
| 4.1  | Topic Generator     | List of topics (title + description)               |
| 4.2  | Brief Writer        | Structured content briefs                          |
| 4.3  | Content Drafting    | First draft articles                               |
| 4.4  | Content Enhancement | Polished, SEO-optimized, publication-ready content |

agents are **composable** — they can call tools, access memory, use documents, or interact with APIs.

#### Concepts

| Concept            | Explanation                                                                            |
| ------------------ | -------------------------------------------------------------------------------------- |
| **LangChain**      | Python framework to build LLM-powered agents that can use tools, memory, and workflows |
| **PromptTemplate** | Structure + variables used to generate consistent outputs from LLMs                    |
| **Tools**          | Functions that your AI agent can call (e.g., search, summarize, recommend)             |
| **Chains**         | Sequences of steps the LLM performs (e.g., extract → summarize → plan)                 |
| **Memory**         | Lets agents remember past interactions                                                 |
| **Agents**         | Smart, autonomous systems that use all the above to complete tasks                     |


## Step 4.1 - AI Agent Core: Topic Generator

In [1]:
import os
import sys
import pandas as pd

In [2]:
# Add project root to sys path
"""
This is needed because this notebook file is under root/notebooks folder. 
Where as the agent files are under root/src/agents folder.
This is to navigate from notebooks to agents folder
"""

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
#print(project_root)
#print(sys.path)

if project_root not in sys.path:
    sys.path.append(project_root)

In [11]:
# Now we can import from src
from src.agents.topic_generator import generate_topics
from src.agents.brief_writer import generate_brief
from src.agents.content_drafter import generate_draft

In [4]:
# Load combined data
data_path = os.path.join(project_root, "data", "combined_data.csv")
#print(data_path)

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data file not found at: {data_path}")

combined_df = pd.read_csv(data_path, parse_dates=["publishedAt"])
print(f"Loaded data from: {data_path}")
print(f"Total articles/ posts: {len(combined_df)}")

combined_df.head()

Loaded data from: /Users/vinothhaldorai/Documents/Vinoth/PROJECTS/content-marketing-agent/data/combined_data.csv
Total articles/ posts: 46


,title,publishedAt,source,publishedDate
0,AI in content marketing: How creators and mark...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
1,AI in Marketing recent news | Content Marketin...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
2,A Complete Guide to Adopting AI in Content Mar...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
3,AI tools for content marketing : r/marketing,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
4,Artificial Intelligence And The Future Of Cont...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08


#### Generate topics using Agent

In [5]:
# Generate topics using agent
# We will pass recent titles from multiple platforms to LLM

titles = combined_df["title"].dropna().tolist()
# print(titles)

# Call the LLM agent
topics = generate_topics(titles)

# Display suggested topics
if isinstance(topics, list):
    print("Topic is returned as list")
    for i, topic in enumerate(topics, 1):
        print(f"Topic {i}: {topic['title']}")
        print(f"Description: {topic['description']}\n")
else:
    # fallback to raw response if JSON parsing failed
    print(topics)

# print(topics)

/Users/vinothhaldorai/Documents/Vinoth/PROJECTS/content-marketing-agent/src/agents/topic_generator.py:58: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/Users/vinothhaldorai/Documents/Vinoth/PROJECTS/content-marketing-agent/src/agents/topic_generator.py:63: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_response = chain.run(keywords=keywords_str)


Topic is returned as list
Topic 1: AI-Powered Content Marketing Revolution
Description: Exploring how creators and marketers are leveraging AI to enhance content strategies and engagement.

Topic 2: The Future of AI Marketing Tools
Description: A comprehensive look at the top AI marketing tools set to transform the industry in 2025.

Topic 3: Automation in Content Creation
Description: Discover the best AI tools for automating content marketing efforts and boosting productivity.

Topic 4: Precision Marketing with AI
Description: How AI is bringing precision to sustainable food marketing and changing consumer interactions.

Topic 5: Affiliate Marketing Meets AI
Description: Strategies for affiliate marketers to utilize AI tools for maximizing client acquisition and revenue.



In [6]:
# Example usage
sample_keywords = ["AI in marketing", "SEO automation", "generative content"]
print(generate_topics(sample_keywords))

[{'title': 'AI-Powered Marketing Strategies', 'description': 'Exploring innovative ways AI is reshaping marketing strategies for businesses.'}, {'title': 'Automating SEO: The Future is Here', 'description': 'A look at how SEO automation is simplifying processes and enhancing search visibility.'}, {'title': 'Creating with AI: The Rise of Generative Content', 'description': 'Understanding how generative content is changing the landscape of content creation in marketing.'}, {'title': 'The Synergy of AI and SEO', 'description': 'How AI technologies are optimizing SEO techniques for better results.'}, {'title': 'Generative Content: Beyond the Hype', 'description': 'Analyzing the practical applications and effectiveness of generative content in marketing campaigns.'}]


## Step 4.2 - Brief Writer Agent

**Objective**

Automatically generate a structured content brief for each topic. This includes:
- Title suggestion
- Section-wise outline
- Tone and style
- Target audience
- Suggested call to action (CTA)

**Concepts**

| Concept                    | What You'll Learn                                           |
| -------------------------- | ----------------------------------------------------------- |
| **Prompt engineering**     | How to guide LLMs to produce structured, consistent outputs |
| **LangChain LLMChain**     | Reusing the same LangChain building block                   |
| **Prompt composition**     | Structuring multi-part outputs cleanly                      |
| **Separation of concerns** | Keeping agents modular, for reuse or replacement            |


In [9]:
# Test the brief_writer agent
sample_topic = "How AI is Revolutionizing Content Marketing"
brief = generate_brief(sample_topic, sample_topic)
print(brief)

**Title:**  
Harnessing the Future: How AI is Revolutionizing Content Marketing

**Outline:**  
1. **Introduction**  
   - Brief overview of the current state of content marketing  
   - Introduction to AI and its growing significance in the industry  
   - Overview of what readers can expect from the article  

2. **Understanding AI in Content Marketing**  
   - Definition of AI and its applications in content marketing  
   - Key technologies involved (e.g., machine learning, natural language processing)  
   - Examples of AI tools currently used in the industry  

3. **AI-Powered Content Creation**  
   - How AI tools assist in generating high-quality content (e.g., articles, blogs, social media posts)  
   - Benefits of AI-generated content (efficiency, personalization, scalability)  
   - Case studies of brands successfully utilizing AI for content creation  

4. **Enhancing Content Strategy with AI**  
   - Utilizing AI for audience analysis and segmentation  
   - Predictive ana

In [10]:
topics = generate_topics(titles)  # list of dicts

for topic in topics:
    brief = generate_brief(title=topic["title"], description=topic["description"])
    print(f"Brief for '{topic['title']}':\n{brief}\n{'-'*40}\n")

Brief for 'AI-Powered Content Creation: Revolutionizing Marketing Strategies':
**Title:**  
AI-Powered Content Creation: Transforming Marketing Strategies for the Future

**Outline:**  
1. **Introduction to AI in Content Creation**  
   - Brief overview of AI technology and its emergence in content creation.  
   - Importance of content in marketing and how AI is changing the landscape.  

2. **The Benefits of AI-Powered Content Creation**  
   - Increased efficiency and time savings for marketers.  
   - Enhanced personalization and targeting capabilities.  
   - Consistency and scalability in content production.  

3. **AI Tools Transforming Content Creation**  
   - Overview of leading AI tools available (e.g., GPT-3, Jasper, Copy.ai).  
   - Case studies of brands successfully using AI for content creation.  

4. **Challenges and Considerations**  
   - Potential pitfalls of relying too heavily on AI (e.g., quality, creativity).  
   - Ethical considerations in AI-generated content

## Step 4.3 - Content Drafting Agent

**Goal**

Turn a topic + brief into a coherent, SEO-friendly first draft blog post.

**Concepts**

We You already generated a brief containing:
- Title
- Outline
- Tone
- Audience
- Call to Action

Now we pass the brief into an LLM to generate a full article draft.

In [12]:
# Test the content drafting agent
brief = {
    "title": "Harnessing AI in Marketing",
    "outline": "1. Introduction\n2. Key AI tools\n3. Personalization\n4. Case Studies\n5. Challenges\n6. Conclusion",
    "tone": "Professional yet approachable",
    "audience": "Digital marketers and content strategists",
    "cta": "Explore our AI content toolkit"
}

draft = generate_draft(brief)
print(draft)

# Harnessing AI in Marketing

## Introduction

In the rapidly evolving digital landscape, artificial intelligence (AI) is no longer a buzzword; it’s a transformative force redefining marketing strategies. As digital marketers and content strategists, embracing AI can enhance efficiency, drive engagement, and ultimately lead to higher conversion rates. This post will explore the key AI tools available, the power of personalization, successful case studies, potential challenges, and how you can harness these technologies to elevate your marketing efforts.

## Key AI Tools

The AI toolkit available to marketers is vast and varied, offering solutions for everything from data analysis to content creation. Here are some essential tools that can revolutionize your marketing strategy:

1. **Chatbots**: Platforms like Drift and Intercom use AI to provide real-time customer service, answering queries and guiding users through the purchasing process, thereby enhancing user experience and increasi

In [17]:
# topics = generate_topics(titles)
briefs = []
for topic in topics:
    title = topic.get("title", "")
    description = topic.get("description", "")
    print(f"Title: {title}")
    print(f"{description}")
    brief = generate_brief(title, description)
    print("**Brief**")
    print(f"{brief}")
    briefs.append(brief)
    
# Generate content for the first brief for testing purposes
#draft = generate_draft(briefs[0])

print(draft)


Title: AI-Powered Content Creation: Revolutionizing Marketing Strategies
Explore how AI tools are enhancing content creation and driving marketing success for brands.
**Brief**
Title:  
Harnessing AI-Powered Content Creation: Transforming Marketing Strategies for the Digital Age

Outline:  
1. **Introduction**  
   - Overview of AI in the marketing landscape  
   - Importance of content in driving engagement and conversions  

2. **Understanding AI-Powered Content Creation**  
   - Definition and explanation of AI content creation tools  
   - Types of AI tools available (e.g., text generation, image creation, video editing)  

3. **Benefits of Using AI in Content Creation**  
   - Increased efficiency and time-saving aspects  
   - Enhanced personalization and targeting capabilities  
   - Improved data analysis for content optimization  

4. **Case Studies: Successful Brands Utilizing AI**  
   - Examples of companies that have successfully integrated AI in their marketing strategies